In [1]:
!pip install scikit-image

^C


   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.9 MB 3.2 MB/s eta 0:00:05
   ---------------------------------------- 0.2/12.9 MB 1.5 MB/s eta 0:00:09
    --------------------------------------- 0.3/12.9 MB 2.1 MB/s eta 0:00:06
    --------------------------------------- 0.3/12.9 MB 1.9 MB/s eta 0:00:07
   - -------------------------------------- 0.4/12.9 MB 1.8 MB/s eta 0:00:07
   - -------------------------------------- 0.5/12.9 MB 1.8 MB/s eta 0:00:07
   - -------------------------------------- 0.6/12.9 MB 1.7 MB/s eta 0:00:08
   - -------------------------------------- 0.6/12.9 MB 1.7 MB/s eta 0:00:08
   -- ------------------------------------- 0.7/12.9 MB 1.8 MB/s eta 0:00:07
   -- ------------------------------------- 0.7/12.9 MB 1.8 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/12.9 MB 1.6 MB/s eta 0:00:08
   -- ------------------------------------- 0.8/12.9 MB 1.5 MB/s eta 0:00:09
   -- 


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
# Daftar jenis acne yang akan diproses
target_acne_classes = ['Pustula', 'papula', 'acne fulminans', 'acne nodules', 'fungal acne']

In [ ]:
# Fungsi untuk ekstraksi fitur GLCM
def glcm_features(image):
    glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    features = []
    # Ekstraksi fitur: Kontras, Homogenitas, Energi, Entropi, dan Korelasi
    features.append(graycoprops(glcm, 'contrast')[0, 0])
    features.append(graycoprops(glcm, 'homogeneity')[0, 0])
    features.append(graycoprops(glcm, 'energy')[0, 0])
    features.append(graycoprops(glcm, 'correlation')[0, 0])
    return features

In [ ]:
# Load dataset dengan debugging
def load_dataset(directory):
    labels = []
    features = []
    for label in os.listdir(directory):
        folder_path = os.path.join(directory, label)
        if os.path.isdir(folder_path) and label in target_acne_classes:
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                
                if image is not None:
                    # Resize dan normalisasi gambar
                    image = cv2.resize(image, (128, 128))
                    
                    # Ekstraksi fitur GLCM
                    feature = glcm_features(image)
                    
                    # Debugging fitur yang diekstraksi
                    print(f"Fitur dari gambar {image_name} dalam kelas {label}: {feature}")
                    
                    features.append(feature)
                    labels.append(label)
    return np.array(features), np.array(labels)

In [ ]:
# Debugging: Memeriksa jumlah kelas dan label unik
def check_classes_and_labels(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    print("Kelas dalam dataset:", unique_labels)
    print("Jumlah sampel per kelas:", counts)

In [ ]:
# Load data training dan testing
train_data, train_labels = load_dataset('path_to_training_folder')
test_data, test_labels = load_dataset('path_to_test_folder')

In [ ]:
# Debugging: Memeriksa apakah data sudah benar
print(f"Shape data latih: {train_data.shape}, Shape label latih: {train_labels.shape}")
check_classes_and_labels(train_labels)

In [ ]:
# Debugging: Memeriksa apakah data uji sudah benar
print(f"Shape data uji: {test_data.shape}, Shape label uji: {test_labels.shape}")
check_classes_and_labels(test_labels)

In [ ]:
# Membagi data training untuk validasi
X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

In [ ]:
# Debugging: Memeriksa hasil split data
print(f"Shape data training setelah split: {X_train.shape}, Shape data validasi: {X_val.shape}")
print(f"Label data training setelah split: {y_train.shape}, Label data validasi: {y_val.shape}")

In [ ]:
# Hyperparameter tuning menggunakan GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=5)
grid.fit(X_train, y_train)

In [ ]:
# Debugging: Menampilkan parameter terbaik yang ditemukan GridSearch
print("Parameter terbaik yang ditemukan oleh GridSearch:", grid.best_params_)

In [ ]:
# Evaluasi dengan data validasi
predictions = grid.predict(X_val)

# Debugging: Menampilkan hasil evaluasi dengan data validasi
print("Hasil evaluasi dengan data validasi:")
print(classification_report(y_val, predictions))
print(f"Accuracy validasi: {accuracy_score(y_val, predictions)}")

In [ ]:
# Evaluasi akhir dengan data testing
test_predictions = grid.predict(test_data)

# Debugging: Menampilkan hasil evaluasi akhir dengan data testing
print("Hasil evaluasi dengan data testing:")
print(classification_report(test_labels, test_predictions))
print(f"Test Accuracy: {accuracy_score(test_labels, test_predictions)}")